In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
from pathlib import Path
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import h5py
import pandas as pd
# Define the project path
PROJECT_PATH = Path('/content/drive/MyDrive/AutomaticHeartSoundClassification-main')
sys.path.append(str(PROJECT_PATH))

DATASET_PATH = Path('/content/drive/MyDrive/AutomaticHeartSoundClassification-main/data')
sys.path.append(str(DATASET_PATH))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DATARAW_PATH = Path('/content/drive/MyDrive/physionet2016')
sys.path.append(str(DATASET_PATH))

In [3]:
MODEL_NAME = "CRNN"      # hoặc "CNN"
TIMESTAMP = "0108_105831"

# Dowload Package

In [4]:
import importlib
import subprocess
import sys

def check_or_install(pkg_name, import_name=None, version=None):
    """
    Check whether a Python package is already installed.
    If the package exists, print its current version.
    If the package does not exist, install the specified version.

    Parameters
    ----------
    pkg_name : str
        Package name as registered on PyPI.
    import_name : str, optional
        Module name used for import (if different from pkg_name).
    version : str, optional
        Required version. If None, any installed version is accepted.
    """
    name = import_name if import_name else pkg_name

    try:
        # Try to import the package
        module = importlib.import_module(name)

        # Retrieve the package version if available
        ver = getattr(module, "__version__", "unknown")

        print(f"[OK] {pkg_name} is already installed (version: {ver})")

    except ImportError:
        # Construct the pip install command
        install_target = pkg_name if version is None else f"{pkg_name}=={version}"

        print(f"[INSTALL] {pkg_name} not found. Installing {install_target} ...")

        # Install the package using pip
        subprocess.check_call([
            sys.executable,
            "-m",
            "pip",
            "install",
            install_target
        ])

In [5]:
#!pip install numpy==1.26.4

In [6]:
# Core ONNX ecosystem (recommended stable versions)
check_or_install("onnx", version="1.17.0")
check_or_install("onnxsim", import_name="onnxsim", version="0.4.33")
check_or_install("onnx-graphsurgeon", import_name="onnx_graphsurgeon")

# Runtime and model conversion tools
check_or_install("onnxruntime", import_name="onnxruntime", version="1.18.1")
check_or_install("onnx2tf", import_name="onnx2tf")
check_or_install("onnxscript ", import_name="onnxscript")
check_or_install("numpy", version="1.26.4")

[OK] onnx is already installed (version: 1.17.0)
[OK] onnxsim is already installed (version: 0.4.33)
[OK] onnx-graphsurgeon is already installed (version: 0.5.8)
[OK] onnxruntime is already installed (version: 1.18.1)
[OK] onnx2tf is already installed (version: 1.28.8)
[OK] onnxscript  is already installed (version: 0.5.7)
[OK] numpy is already installed (version: 1.26.4)


In [7]:
# TensorFlow and Keras (stable and compatible versions)
check_or_install("tensorflow", import_name="tensorflow", version="2.19.0")
check_or_install("tf-keras", import_name="tf_keras", version="2.19.0")

# Auxiliary packages for signal processing and edge deployment
check_or_install("h5py")
check_or_install("librosa")
check_or_install("ai_edge_litert", version="1.2.0")
check_or_install("sng4onnx")

[OK] tensorflow is already installed (version: 2.19.0)
[OK] tf-keras is already installed (version: 2.19.0)
[OK] h5py is already installed (version: 3.15.1)
[OK] librosa is already installed (version: 0.11.0)
[OK] ai_edge_litert is already installed (version: 1.2.0)
[OK] sng4onnx is already installed (version: 1.0.4)


In [8]:
import onnx
print("ONNX version:", onnx.__version__)
print("ONNX file:", onnx.__file__)

ONNX version: 1.17.0
ONNX file: /usr/local/lib/python3.12/dist-packages/onnx/__init__.py


# Loss and Metrics

In [9]:
import torch.nn.functional as F


def nll_loss(output, target):
    return F.nll_loss(output, target)

def ce_loss(output, target):
    return F.cross_entropy(output, target)

In [10]:
import torch


def accuracy(output, target):
    with torch.no_grad():
        pred = torch.argmax(output, dim=1)
        assert pred.shape[0] == len(target)
        correct = 0
        correct += torch.sum(pred == target).item()
    return correct / len(target)


def top_k_acc(output, target, k=3):
    with torch.no_grad():
        pred = torch.topk(output, k, dim=1)[1]
        assert pred.shape[0] == len(target)
        correct = 0
        for i in range(k):
            correct += torch.sum(pred[:, i] == target).item()
    return correct / len(target)

In [11]:
MODEL_DIR_MAP = {
    "VGG11": "VGG11",
    "CNN": "simple_cnn",
    "CRNN": "CRNN",
    "LSTM": "LSTM",
}


# Config Parse

In [12]:
import os
import logging
from pathlib import Path
from datetime import datetime
import argparse
import collections
import importlib.util

class ConfigParser:
    def __init__(self, config_dict, resume=None, modification=None, run_id=None):
        self._config = config_dict
        self.resume = resume

        # Apply CLI modifications
        if modification:
            for key_path, value in modification.items():
                keys = key_path.split(';')
                d = self._config
                for k in keys[:-1]:
                    d = d[k]
                d[keys[-1]] = value

        # Setup save and log directories
        save_dir = Path(self.config['trainer']['save_dir'])
        exper_name = self.config['name']
        if run_id is None:
            run_id = datetime.now().strftime(r'%m%d_%H%M%S')
        self._save_dir = save_dir / 'models' / exper_name / run_id
        self._log_dir = save_dir / 'log' / exper_name / run_id

        self.save_dir.mkdir(parents=True, exist_ok=True)
        self.log_dir.mkdir(parents=True, exist_ok=True)

        # Optional: setup logging
        # setup_logging(self.log_dir)

    @classmethod
    def from_args(cls, parser, options=None):
        """
        parser: argparse.ArgumentParser đã được add_argument xong
        options: list các CustomArgs để override
        """
        if options is None:
            options = []

        # Parse arguments (an toàn cho cả terminal và notebook)
        args = parser.parse_args()

        # Xử lý device
        if args.device is not None:
            os.environ["CUDA_VISIBLE_DEVICES"] = args.device

        # Xử lý resume
        if args.resume is not None:
            resume = Path(args.resume)
            cfg_fname = resume.parent / 'config.py'  # hoặc .json nếu bạn dùng cả 2
        else:
            resume = None
            assert args.config is not None, "Phải chỉ định -c config.py"
            cfg_fname = Path(args.config)

        # Đọc config từ file .py
        if not cfg_fname.exists():
            raise FileNotFoundError(f"Không tìm thấy config file: {cfg_fname}")

        if cfg_fname.suffix == '.py':
            spec = importlib.util.spec_from_file_location("config_module", cfg_fname)
            config_module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(config_module)
            if not hasattr(config_module, 'config'):
                raise ValueError(f"File {cfg_fname} phải định nghĩa biến 'config'")
            config_dict = config_module.config
        else:
            raise ValueError("Chỉ hỗ trợ file config .py")

        # Parse custom CLI overrides
        CustomArgs = collections.namedtuple('CustomArgs', 'flags type target')
        modification = {}
        for opt in options:
            for flag in opt.flags:
                arg_name = flag.lstrip('-').replace('-', '_')
                if hasattr(args, arg_name) and getattr(args, arg_name) is not None:
                    modification[opt.target] = opt.type(getattr(args, arg_name))

        return cls(config_dict, resume, modification)

    def init_obj(self, name, module, *args, **kwargs):
        module_name = self[name]['type']
        module_args = dict(self[name]['args'])
        module_args.update(kwargs)
        if isinstance(module, dict):
            obj_class = module[module_name]
        else:
            obj_class = getattr(module, module_name)

        return obj_class(*args, **module_args)

    def init_ftn(self, name, module, *args, **kwargs):
        from functools import partial
        module_name = self[name]['type']
        module_args = dict(self[name]['args'])
        module_args.update(kwargs)
        if isinstance(module, dict):
            obj_class = module[module_name]
        else:
            obj_class = getattr(module, module_name)

        return obj_class(*args, **module_args)

    def __getitem__(self, name):
        return self.config[name]

    def get_logger(self, name, verbosity=2):
        logger = logging.getLogger(name)
        logger.setLevel({0: logging.WARNING, 1: logging.INFO, 2: logging.DEBUG}[verbosity])
        return logger

    @property
    def config(self):
        return self._config

    @property
    def save_dir(self):
        return self._save_dir

    @property
    def log_dir(self):
        return self._log_dir

# Model

In [13]:
import torch.nn as nn
import numpy as np
from abc import abstractmethod


class BaseModel(nn.Module):
    """
    Base class for all models
    """
    @abstractmethod
    def forward(self, *inputs):
        """
        Forward pass logic

        :return: Model output
        """
        raise NotImplementedError

    def __str__(self):
        """
        Model prints with number of trainable parameters
        """
        model_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in model_parameters])
        return super().__str__() + '\nTrainable parameters: {}'.format(params)

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# functions of initializing layers
def init_layer(layer):
    """Initialize a Linear or Convolutional layer."""
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)

def init_bn(bn):
    """Initialize a Batchnorm layer."""
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)

def init_rnn(rnn):
    """init_rnn
    Initialized RNN weights, independent of type GRU/LSTM/RNN
    :param rnn: the rnn model
    """
    for name, param in rnn.named_parameters():
        if 'bias' in name:
            nn.init.constant_(param, 0.0)
        elif 'weight' in name:
            nn.init.xavier_uniform_(param)

def reset_parameters(model):
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            init_layer(module)
        elif isinstance(module, nn.Linear):
            init_layer(module)
        elif isinstance(module, nn.BatchNorm2d):
            init_bn(module)
        elif isinstance(module, nn.LSTM):
            init_rnn(module)


## CNN

In [15]:
class simple_cnn(BaseModel):
    def __init__(self, num_classes = 2, in_channel=1):
        super(simple_cnn, self).__init__()
        self.in_channel = in_channel
        self.conv1 = nn.Conv2d(in_channel,16,3)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16,32,3)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32,64,3)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Linear(64,32)
        self.output = nn.Linear(32,num_classes)
        self.activate = nn.Softmax(dim=1)

        self.init_weights()

    def init_weights(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_layer(self.conv3)
        init_layer(self.fc1)
        init_layer(self.output)
        init_bn(self.bn1)
        init_bn(self.bn2)
        init_bn(self.bn3)


    def forward(self,x):
        B, mel_bins, num_frames = x.size()
        x = x.view(B, self.in_channel, -1, num_frames)
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = self.bn1(x)
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = self.bn2(x)
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        x = self.bn3(x)
        x = self.pool(x).reshape(x.size(0),-1)
        x = self.fc1(x)
        x = F.dropout(x, p=0.5, training=self.training)
        out = self.output(x)
        out = self.activate(out)
        return out

## Deeper CNN model, VGG like structure

In [16]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = in_channels,
                               out_channels = out_channels,
                               kernel_size = (3,3),
                               stride = (1,1),
                               padding = (1,1),
                               bias =False)

        self.conv2 = nn.Conv2d(in_channels = out_channels,
                               out_channels = out_channels,
                               kernel_size = (3,3),
                               stride = (1,1),
                               padding = (1,1),
                               bias = False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weights()

    def init_weights(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='max', activation = 'relu'):
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        if activation == 'relu':
            x = F.relu_(self.bn2(self.conv2(x)))
        elif activation == 'sigmoid':
            x = torch.sigmoid(self.bn2(self.conv2(x)))

        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x

class VGG_11(BaseModel):
    def __init__(self, num_classes, in_channel):
        super(VGG_11, self).__init__()
        self.in_channel = in_channel
        self.bn0 = nn.BatchNorm2d(128)
        self.conv1 = ConvBlock(in_channels=in_channel, out_channels=64)
        self.conv2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv4 = ConvBlock(in_channels=256, out_channels=512)
        self.fc_final = nn.Linear(512, num_classes)
        self.init_weights()

    def init_weights(self):
        init_bn(self.bn0)
        init_layer(self.fc_final)

    def forward(self, input):
        # (batch_size, 3, mel_bins, time_stamps)
        B, mel_bins, num_frames = input.size()
        x = input.view(B, self.in_channel, -1, num_frames)
        x = x.transpose(1, 2)
        x = self.bn0(x)
        x = x.transpose(1,2)

        # (samples_num, channel, mel_bins, time_stamps)
        x = self.conv1(x, pool_size=(2, 2), pool_type='max')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, pool_size=(2, 2), pool_type='max')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv3(x, pool_size=(2, 2), pool_type='max')
        x = F.dropout(x, p=0.2, training = self.training)
        x = self.conv4(x, pool_size=(2, 2), pool_type='max')
        x = F.dropout(x, p=0.2, training=self.training)

        output = F.max_pool2d(x, kernel_size=x.shape[2:])
        output = output.view(output.shape[0:2])
        output = F.log_softmax(self.fc_final(output), dim=-1)
        return output


class VGG_13(BaseModel):
    def __init__(self, num_classes, in_channel):
        super(VGG_13, self).__init__()
        self.in_channel = in_channel
        self.bn0 = nn.BatchNorm2d(128)
        self.conv1 = ConvBlock(in_channels=in_channel, out_channels=64)
        self.conv2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv5 = ConvBlock(in_channels=512, out_channels=512)
        self.fc_1 = nn.Linear(512 * 4* 10, 4096)
        self.fc_2 = nn.Linear(4096, 4096)
        self.fc_final = nn.Linear(4096, num_classes)
        self.init_weights()

    def init_weights(self):
        init_bn(self.bn0)
        init_layer(self.fc_final)
        init_layer(self.fc_1)
        init_layer(self.fc_2)

    def forward(self, input):
        # (batch_size, 3, mel_bins, time_stamps)
        B, mel_bins, num_frames = input.size()
        x = input.view(B, self.in_channel, -1, num_frames)
        x = x.transpose(1, 2)
        x = self.bn0(x)
        x = x.transpose(1,2)

        # (samples_num, channel, time_stemps, mel_bins)
        x = self.conv1(x, pool_size=(2, 2), pool_type='max')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, pool_size=(2, 2), pool_type='max')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv3(x, pool_size=(2, 2), pool_type='max')
        x = F.dropout(x, p=0.2, training = self.training)
        x = self.conv4(x, pool_size=(2, 2), pool_type='max')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv5(x, pool_size=(2, 2), pool_type='max')
        x = F.dropout(x, p=0.2, training=self.training)
        #output = F.max_pool2d(x, kernel_size=x.shape[2:])
        #output = output.view(output.shape[0:2])
        x = x.view(x.shape[0], -1)
        x = F.relu_(self.fc_1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc_2(x))
        x = F.dropout(x, p=0.5, training=self.training)
        output = F.log_softmax(self.fc_final(x),dim=-1)
        #output = F.log_softmax(self.fc_final(output), dim=-1)
        return output

## RNN - BiLTSM

In [17]:
class BiLSTM(BaseModel):
    def __init__(self,
                 input_dim=200,
                 hidden_dim=256,
                 num_layers =2,
                 dropout=0.2,
                 num_classes=2,
                 pooling='first',
                 model='lstm',
                 BN=False):
        super(BiLSTM,self).__init__()
        if model == 'lstm':
            self.LSTM = nn.LSTM(input_size=input_dim,
                                hidden_size =hidden_dim,
                                num_layers =num_layers,
                                batch_first =True,
                                dropout=dropout,
                                bidirectional=True)
        elif model == 'gru':
            self.LSTM = nn.GRU(input_size=input_dim,
                               hidden_size =hidden_dim,
                               num_layers =num_layers,
                               batch_first =True,
                               dropout=dropout,
                               bidirectional=True)
        init_rnn(self.LSTM)
        self.BN = BN
        if self.BN:
            self.BatchNorm=nn.BatchNorm1d(hidden_dim*2)

        self.layer_out = nn.Linear(hidden_dim*2,num_classes,bias=False)
        self.pooling=pooling

    def forward(self,x):
        x = x.transpose(2,1) #[Batchsize x Time_frames x Mel_bin]
        x,_ = self.LSTM(x)
        if self.BN:
            x = self.BatchNorm(x.transpose(1,2))
            x = x.transpose(1,2)
        dim =1
        x = self.layer_out(x)  #200,2
        if self.pooling == 'avg':
            x = x.mean(dim)   #average pooling
        elif self.pooling == 'first':
            x = x.select(dim,0)  #first time step
        elif self.pooling == 'last':
            x = x.select(dim,-1)  #first time step
        elif self.pooling == 'max':
            x = x.max(dim)[0]
        elif  self.pooling == 'linear':
            (x**2).sum(dim) / x.sum(dim)
        elif self.pooling == 'exp':
            (x.exp() * x).sum(dim) / x.exp().sum(dim)

        # print(out.shape)
        # out = self.LogSoftmax(out)#the input given is expected to contain log-probabilities. Obtaining log-probabilities in a neural network is easily achieved by adding a LogSoftmax layer in the last layer of your network.
        return x#,A

## CRNN

In [18]:
class CNN_Encoder(nn.Module):
    def __init__(self, in_channels):
        super(CNN_Encoder, self).__init__()

        self.conv1 = ConvBlock(in_channels=in_channels, out_channels=32)
        self.conv2 = ConvBlock(in_channels=32, out_channels=64)
        self.conv3 = ConvBlock(in_channels=64, out_channels=128)
        self.conv4 = ConvBlock(in_channels=128, out_channels=256)
    def forward(self, input):
        x = input
        x = self.conv1(x, pool_size=(2, 2))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, pool_size=(2, 2))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv3(x, pool_size=(2, 2))
        x = F.dropout(x, p=0.2, training = self.training)
        x = self.conv4(x, pool_size=(2, 2))
        x = F.dropout(x, p=0.2, training=self.training)

        return x

class crnn(BaseModel):
    def __init__(self, in_channel,
                 hidden_dim=128,
                 num_layers = 2,
                 dropout = 0.2,
                 pooling = 'first',
                 model='lstm',
                 BN = False,
                 num_classes=2):
        super(crnn,self).__init__()

        self.in_channel = in_channel
        #self.bn0 = nn.BatchNorm2d(128)
        self.bn0 = None

        self.cnn = CNN_Encoder(in_channels=in_channel)
        self.bilstm = BiLSTM(input_dim=256,
                             hidden_dim=hidden_dim,
                             num_layers =num_layers,
                             dropout =dropout,
                             num_classes=num_classes,
                             pooling=pooling,
                             model=model,
                             BN=BN)


    def forward(self,input):
        # (batch_size, 3, mel_bins, time_stamps)
        B, mel_bins, num_frames = input.size()
        #x = input.view(B, self.in_channel, -1, num_frames)
        #x = x.transpose(1, 2)
        x = input.view(B, self.in_channel, mel_bins // self.in_channel, num_frames)

        x = self.cnn(x)
        #x = F.max_pool2d(x,kernel_size=(x.size(-2), 1)) # pool mel dimension
        x = F.adaptive_max_pool2d(x, (1, None))
        x = x.squeeze(-2)  # [B x C x Time_frames]

        output = self.bilstm(x)
        return output

# Load Model

In [19]:
from pathlib import Path

def get_model_best_path(project_path, model_name, timestamp):
    """
    Trả về đường dẫn tới model_best.pth theo cấu trúc chuẩn:
    saved/<model_dir>/models/Physionet_<model_dir>/<timestamp>/model_best.pth
    """

    if model_name not in MODEL_DIR_MAP:
        raise ValueError(f"Unsupported model name: {model_name}")

    model_dir = MODEL_DIR_MAP[model_name]

    return (
        project_path
        / "saved"
        / model_dir
        / "models"
        / f"Physionet_{model_dir}"
        / timestamp
        / "model_best.pth"
    )


In [20]:
model_path = get_model_best_path(
    project_path=PROJECT_PATH,
    model_name=MODEL_NAME,
    timestamp=TIMESTAMP
)

print(model_path)

/content/drive/MyDrive/AutomaticHeartSoundClassification-main/saved/CRNN/models/Physionet_CRNN/0108_105831/model_best.pth


In [21]:
if model_path.exists():
    size_bytes = model_path.stat().st_size

    # Convert to human-readable format
    if size_bytes < 1024:
        size_str = f"{size_bytes} Bytes"
    elif size_bytes < 1024**2:
        size_str = f"{size_bytes / 1024:.2f} KB"
    elif size_bytes < 1024**3:
        size_str = f"{size_bytes / (1024**2):.2f} MB"
    else:
        size_str = f"{size_bytes / (1024**3):.2f} GB"

    print(f"✅ File exists!")
    print(f"   Size: {size_str} ({size_bytes:,} bytes)")
else:
    print("❌ File does NOT exist at the specified path.")
    print("   Possible reasons:")
    print("   - Training did not complete or model_best.pth was not saved")
    print("   - Wrong timestamp (check your actual training time)")
    print("   - Model saved elsewhere (e.g., in TFLITE folder)")
    print()
    print("Searching for all .pth files in the project for suggestions...")

    # Search for any .pth files in the entire project
    pth_files = sorted(PROJECT_PATH.rglob("*.pth"))
    if pth_files:
        print("\nFound .pth files:")
        for p in pth_files[:10]:  # Show max 10 files
            rel_size = p.stat().st_size / (1024**2)
            print(f"   → {p.relative_to(PROJECT_PATH)}  ({rel_size:.2f} MB)")
        if len(pth_files) > 10:
            print(f"   ... and {len(pth_files)-10} more files")
    else:
        print("   No .pth files found in the entire project!")

✅ File exists!
   Size: 58.37 MB (61,207,937 bytes)


In [22]:
MODEL_CLASS_MAP = {
    "VGG11": {
        "class": VGG_11,
        "in_channel": 3,
        "num_classes": 2
    },
    "CNN": {
        "class": simple_cnn,
        "in_channel": 3,
        "num_classes": 2
    },
    "CRNN": {
        "class": crnn,
        "in_channel": 1,
        "num_classes": 2,
        # Các tham số mặc định cho CRNN (có thể chỉnh nếu cần)
        "extra_kwargs": {
            "hidden_dim": 256,
            "num_layers": 2,
            "dropout": 0.2,
            "pooling": "first",
            "model": "lstm",
            "BN": False
        }
    },
    "LSTM": {
        "class": BiLSTM,
        "in_channel": None,  # Không dùng in_channel
        "num_classes": 2,
        "extra_kwargs": {
            "input_dim": 128,      # Thường là số mel bins sau khi pool mel dim
            "hidden_dim": 256,
            "num_layers": 2,
            "dropout": 0.2,
            "pooling": "first",
            "model": "lstm",
            "BN": False
        }
    }
}

In [23]:
def build_model_for_convert(model_name: str, cfg: dict):
    ModelClass = cfg["class"]
    model_cfg = cfg["model"]
    input_cfg = cfg["input"]

    # ===== CRNN =====
    if model_name == "CRNN":
        model = ModelClass(
            in_channel=input_cfg["in_channel"],
            hidden_dim=model_cfg["hidden_dim"],
            num_layers=model_cfg["num_layers"],
            dropout=model_cfg["dropout"],
            pooling=model_cfg["pooling"],
            model=model_cfg["rnn_type"],
            BN=model_cfg["BN"],
            num_classes=model_cfg["num_classes"]
        )

    # ===== LSTM =====
    elif model_name == "LSTM":
        model = ModelClass(
            input_dim=input_cfg["mel_bins"],
            hidden_dim=model_cfg["hidden_dim"],
            num_layers=model_cfg["num_layers"],
            dropout=model_cfg["dropout"],
            num_classes=model_cfg["num_classes"]
        )

    # ===== CNN / VGG =====
    else:
        model = ModelClass(
            in_channel=input_cfg["in_channel"],
            num_classes=model_cfg["num_classes"]
        )

    model.eval()
    return model


In [61]:
# =========================
# CHECK MODEL NAME
# =========================
if MODEL_NAME not in MODEL_CLASS_MAP:
    raise ValueError(
        f"Model {MODEL_NAME} không được hỗ trợ. "
        f"Các model hỗ trợ: {list(MODEL_CLASS_MAP.keys())}"
    )

cfg = MODEL_CLASS_MAP[MODEL_NAME]
ModelClass = cfg["class"]

# =========================
# BUILD MODEL (THEO KIẾN TRÚC)
# =========================
if MODEL_NAME == "CRNN":
    model = ModelClass(
        in_channel=cfg["in_channel"],
        num_classes=cfg["num_classes"],
        **cfg.get("extra_kwargs", {})
    )

elif MODEL_NAME == "LSTM":
    # LSTM ở đây là BiLSTM, không dùng in_channel
    model = ModelClass(
        num_classes=cfg["num_classes"],
        **cfg.get("extra_kwargs", {})
    )

elif MODEL_NAME in ["VGG11", "CNN"]:
    model = ModelClass(
        in_channel=cfg["in_channel"],
        num_classes=cfg["num_classes"]
    )

else:
    raise ValueError("Cấu hình model không hợp lệ")

model.to(device)
model.eval()

crnn(
  (cnn): CNN_Encoder(
    (conv1): ConvBlock(
      (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv2): ConvBlock(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): ConvBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3),

In [62]:
# =========================
# LOAD WEIGHTS (FINAL – STRICT & SAFE)
# =========================

checkpoint = torch.load(model_path, map_location=device, weights_only=False)

# ---- extract state_dict ----
if isinstance(checkpoint, dict):
    state_dict = checkpoint.get("state_dict", checkpoint.get("model", checkpoint))
else:
    state_dict = checkpoint.state_dict() if hasattr(checkpoint, "state_dict") else checkpoint

# ---- remove DataParallel prefix ----
if all(k.startswith("module.") for k in state_dict.keys()):
    from collections import OrderedDict
    state_dict = OrderedDict((k[7:], v) for k, v in state_dict.items())

# ---- remove BiLSTM BatchNorm (BN=False) ----
if MODEL_NAME in ["CRNN", "LSTM"]:
    if hasattr(model, "bilstm") and hasattr(model.bilstm, "BN") and model.bilstm.BN is False:
        bn_keys = [k for k in state_dict.keys() if "BatchNorm" in k]
        for k in bn_keys:
            del state_dict[k]

# ---- remove CRNN input BatchNorm (bn0) ----
if MODEL_NAME == "CRNN":
    bn0_keys = [k for k in state_dict.keys() if k.startswith("bn0.")]
    for k in bn0_keys:
        del state_dict[k]

# ---- load strictly ----
model.load_state_dict(state_dict, strict=True)
model.to(device)
model.eval()

print(f"✅ Đã load model {MODEL_NAME} thành công từ: {model_path}")


✅ Đã load model CRNN thành công từ: /content/drive/MyDrive/AutomaticHeartSoundClassification-main/saved/CRNN/models/Physionet_CRNN/0108_105831/model_best.pth


# Punning

In [63]:
import torch.nn.utils.prune as prune

parameters_to_prune = []
for name, module in model.named_modules():
    if isinstance(module, (torch.nn.Conv2d, torch.nn.Linear)):
        parameters_to_prune.append((module, 'weight'))
        # Optional: prune bias too
        # if module.bias is not None:
        #     parameters_to_prune.append((module, 'bias'))

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.5,  # 50% weights
)

print("✅ Đã apply pruning (với mask)")

✅ Đã apply pruning (với mask)


In [64]:
features_file = DATASET_PATH / "logmel_features.h5"
label_file = DATASET_PATH / "label.csv"

In [65]:
import librosa
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

EPS = 1e-8

def standard_normal_variate(data):
    """Z-score normalization trên toàn bộ feature"""
    mean = np.mean(data)
    std = np.std(data) + EPS
    return (data - mean) / std

class LogMelDataset(Dataset):
    def __init__(self, features_h5_path, label_csv_path, keys=None,
                 delta=True, norm=True, duration=5, hop_length=15, training=True):
        self.features_h5 = h5py.File(features_h5_path, 'r')
        self.labels_df = pd.read_csv(label_csv_path)

        if keys is None:
            self.keys = list(self.features_h5.keys())
        else:
            self.keys = keys

        # Map key (filename stem) → label
        self.key_to_label = dict(zip(self.labels_df.iloc[:, 0].astype(str), self.labels_df.iloc[:, 1]))

        self.delta = delta
        self.norm = norm
        self.duration = duration
        self.hop_length = hop_length
        self.training = training

        # Độ dài cố định: 5 giây với hop 15ms → 5000 / 15 = 333.333 → lấy 333 frames
        self.fixed_length = int(self.duration * 1000 / self.hop_length)  # 333

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        feature = self.features_h5[key][()]  # shape: (mel_bins, time_frames), thường là (128, T)

        # Chuẩn hóa Z-score nếu bật
        if self.norm:
            feature = standard_normal_variate(feature)

        # Đảm bảo luôn là 3D: (C, mel_bins, T)
        if feature.ndim == 2:
            feature = feature[np.newaxis, :, :]  # (1, 128, T)

        channels, mel_bins, num_frames = feature.shape

        # Thêm delta + delta-delta nếu bật
        if self.delta:
            orig = feature[0]  # (128, T)
            delta1 = librosa.feature.delta(orig)
            delta2 = librosa.feature.delta(delta1)
            feature = np.stack([orig, delta1, delta2], axis=0)  # (3, 128, T)
            channels = 3

        # Crop hoặc pad để đạt đúng fixed_length (333 frames)
        if num_frames >= self.fixed_length:
            if self.training:
                start = random.randint(0, num_frames - self.fixed_length)
            else:
                start = (num_frames - self.fixed_length) // 2
            feature = feature[:, :, start:start + self.fixed_length]
        else:
            pad_width = self.fixed_length - num_frames
            feature = np.pad(feature, ((0, 0), (0, 0), (0, pad_width)), mode='wrap')

        # Bây giờ feature chắc chắn là (channels, 128, 333)
        feature_tensor = torch.from_numpy(feature).float()

        # Quan trọng: reshape để phù hợp với model VGG_11 gốc
        if channels > 1:  # delta=True → ghép 3 channel thành 384 mel bins
            feature_tensor = feature_tensor.view(channels * mel_bins, self.fixed_length)  # (384, 333)
        else:
            feature_tensor = feature_tensor.squeeze(0)  # (128, 333)

        label = torch.tensor(self.key_to_label.get(key, 0), dtype=torch.long)
        return feature_tensor, label

    def close(self):
        if hasattr(self, 'features_h5'):
            self.features_h5.close()

In [66]:
# ================== CHIA TRAIN/VAL KEYS ==================
# Load toàn bộ keys từ file .h5
with h5py.File(features_file, 'r') as hf:
    all_keys = list(hf.keys())

# Load labels để stratify (giữ tỷ lệ lớp khi chia train/val)
labels_all = pd.read_csv(label_file).iloc[:, 1].values  # cột label (thường là cột thứ 2)

# Chia 80/20, giữ nguyên tỷ lệ nhãn (rất quan trọng cho dataset imbalance như PhysioNet)
train_keys, val_keys = train_test_split(
    all_keys,
    test_size=0.2,
    random_state=42,
    stratify=labels_all
)

print(f"Total samples: {len(all_keys)}")
print(f"Train samples: {len(train_keys)}, Val samples: {len(val_keys)}")

Total samples: 3240
Train samples: 2592, Val samples: 648


In [67]:
train_dataset = LogMelDataset(
    features_file,
    label_file,
    keys=train_keys,
    delta=True,      # phải bật vì model train với delta
    norm=True,
    duration=5,
    hop_length=15,
    training=True
)

val_dataset = LogMelDataset(
    features_file,
    label_file,
    keys=val_keys,
    delta=True,
    norm=True,
    duration=5,
    hop_length=15,
    training=False
)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,        # giảm xuống 2 để tránh warning
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")
print("Mỗi sample có shape cố định → không cần padding trong collate")

Train samples: 2592, Val samples: 648
Mỗi sample có shape cố định → không cần padding trong collate


In [68]:
'''
model.train()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
criterion = torch.nn.NLLLoss()  # vì model output log_softmax

num_finetune_epochs = 10

for epoch in range(num_finetune_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Fine-tune Epoch {epoch+1}/{num_finetune_epochs} - Avg Loss: {avg_loss:.6f}")

    # Đánh giá trên validation (tùy chọn)
    if (epoch + 1) % 5 == 0 or epoch == num_finetune_epochs - 1:
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                pred = torch.argmax(outputs, dim=1)
                correct += (pred == labels).sum().item()
                total += labels.size(0)
        acc = correct / total if total > 0 else 0
        print(f"   >>> Val Accuracy sau epoch {epoch+1}: {acc:.4f}")
        model.train()

# Đóng dataset để giải phóng file handle
train_dataset.close()
val_dataset.close()

model.eval()
print("Fine-tune sau pruning hoàn tất")
'''

'\nmodel.train()\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)\ncriterion = torch.nn.NLLLoss()  # vì model output log_softmax\n\nnum_finetune_epochs = 10\n\nfor epoch in range(num_finetune_epochs):\n    model.train()\n    running_loss = 0.0\n\n    for inputs, labels in train_loader:\n        inputs, labels = inputs.to(device), labels.to(device)\n\n        optimizer.zero_grad()\n        outputs = model(inputs)\n        loss = criterion(outputs, labels)\n        loss.backward()\n        optimizer.step()\n\n        running_loss += loss.item()\n\n    avg_loss = running_loss / len(train_loader)\n    print(f"Fine-tune Epoch {epoch+1}/{num_finetune_epochs} - Avg Loss: {avg_loss:.6f}")\n\n    # Đánh giá trên validation (tùy chọn)\n    if (epoch + 1) % 5 == 0 or epoch == num_finetune_epochs - 1:\n        model.eval()\n        correct = 0\n        total = 0\n        with torch.no_grad():\n            for inputs, labels in val_loader:\n             

In [69]:
# ================== LÀM PRUNING PERMANENT ==================
import torch.nn.utils.prune as prune  # nếu chưa import

for module, name in parameters_to_prune:
    prune.remove(module, name)

print("✅ Đã remove pruning mask → pruning permanent")

# ================== LƯU MODEL PRUNED ==================
PRUNED_PATH = PROJECT_PATH / "TFLITE" / f"{MODEL_NAME.lower()}_{TIMESTAMP}_pruned_finetuned.pth"
torch.save(model.state_dict(), PRUNED_PATH)
print(f"✅ Model pruned + finetuned lưu tại: {PRUNED_PATH}")

# Đóng h5 file
train_dataset.close()

✅ Đã remove pruning mask → pruning permanent
✅ Model pruned + finetuned lưu tại: /content/drive/MyDrive/AutomaticHeartSoundClassification-main/TFLITE/crnn_0108_105831_pruned_finetuned.pth


## Define input shape & Dummy input

In [70]:
# ================== DEFINE INPUT SHAPE & DUMMY INPUT ==================
# Input shape cố định sau khi fix dataset: (batch, 384, 333)
dummy_input = torch.randn(1, 384, 333).to(device)

# Chuyển sang ONXX

In [71]:
# Export
ONNX_PATH = PROJECT_PATH / "TFLITE" / f"{MODEL_NAME.lower()}_{TIMESTAMP}_final.onnx"
ONNX_PATH.parent.mkdir(parents=True, exist_ok=True)
print(ONNX_PATH)

/content/drive/MyDrive/AutomaticHeartSoundClassification-main/TFLITE/crnn_0108_105831_final.onnx


In [78]:
print("🔧 Flattening LSTM parameters...")

if hasattr(model, 'bilstm') and hasattr(model.bilstm, 'LSTM'):
    model.bilstm.LSTM.flatten_parameters()
    print("✅ LSTM flattened\n")
else:
    print("⚠️  No LSTM found\n")

🔧 Flattening LSTM parameters...
✅ LSTM flattened



In [79]:
original_forward = model.forward

def export_forward_simple_cnn(input):
    B, mel_bins, num_frames = input.size()

    x = input.view(B, model.in_channel, -1, num_frames)

    x = F.relu(model.bn1(model.conv1(x)))
    x = F.max_pool2d(x, 2)

    x = F.relu(model.bn2(model.conv2(x)))
    x = F.max_pool2d(x, 2)

    x = F.relu(model.bn3(model.conv3(x)))
    x = F.max_pool2d(x, 2)

    x = model.pool(x)
    x = x.view(B, -1)

    x = F.relu(model.fc1(x))
    output = model.output(x)   # Softmax có thể bỏ ở edge
    return output

def export_forward_vgg(input):
    B, mel_bins, num_frames = input.size()
    x = input.view(B, model.in_channel, -1, num_frames)
    x = x.transpose(1, 2)
    x = model.bn0(x)
    x = x.transpose(1, 2)

    x = model.conv1(x, pool_size=(2, 2), pool_type='max')

    x = nn.AdaptiveMaxPool2d((1, 1))(x)
    x = x.view(B, -1)
    output = model.fc_final(x)
    return output

def export_forward_crnn(input):
    """
    ONNX-compatible forward pass
    Uses fixed kernel size for pooling instead of adaptive pooling
    """
    B, mel_bins, num_frames = input.size()

    # Reshape
    x = input.view(B, model.in_channel, mel_bins // model.in_channel, num_frames)

    # CNN encoder - this includes 4 ConvBlocks with pooling
    x = model.cnn(x)

    # Pool frequency dimension using FIXED kernel size
    # This replaces F.adaptive_max_pool2d(x, (1, None))
    x = torch.nn.functional.max_pool2d(
        x,
        kernel_size=(freq_dim_after_cnn, 1)  # Use calculated size
    )
    x = x.squeeze(2)  # Remove frequency dimension

    # BiLSTM
    output = model.bilstm(x)

    return output



if MODEL_NAME == "CNN":
    model.forward = export_forward_simple_cnn
elif MODEL_NAME == "VGG11":
    model.forward = export_forward_vgg
elif MODEL_NAME == "CRNN":
    model.forward = export_forward_crnn
else:
    raise ValueError("Unsupported model")

In [36]:
'''
try:
    torch.onnx.export(
        model,
        dummy_input,
        str(ONNX_PATH),
        export_params=True,
        opset_version=17,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size"},
            "output": {0: "batch_size"}
        },
        dynamo=False
        # verbose=False
    )
    print(f"✅ Export ONNX thành công: {ONNX_PATH}")
except Exception as e:
    print("Lỗi export:", e)
finally:
    model.forward = original_forward
    print("Đã khôi phục forward gốc")

# Kiểm tra file
import os
if os.path.exists(ONNX_PATH):
    print(f"Size: {os.path.getsize(ONNX_PATH) / (1024**2):.2f} MB")
'''

✅ Export ONNX thành công: /content/drive/MyDrive/AutomaticHeartSoundClassification-main/TFLITE/crnn_0108_105831_final.onnx
Đã khôi phục forward gốc
Size: 14.52 MB


In [80]:
try:
    print(f"📦 Exporting to: {ONNX_PATH.name}\n")

    torch.onnx.export(
        model,
        dummy_input,
        str(ONNX_PATH),

        export_params=True,
        opset_version=17,
        do_constant_folding=True,

        input_names=["input"],
        output_names=["output"],

        dynamic_axes={
            "input": {0: "batch_size"},
            "output": {0: "batch_size"}
        },

        dynamo=False,
        training=torch.onnx.TrainingMode.EVAL,
        keep_initializers_as_inputs=False,
        verbose=False
    )

    print(f"✅ ONNX export successful!\n")

except Exception as e:
    print(f"❌ Export failed: {e}\n")
    import traceback
    traceback.print_exc()
    raise

finally:
    model.forward = original_forward
    print("✅ Restored original forward\n")

📦 Exporting to: crnn_0108_105831_final.onnx

✅ ONNX export successful!

✅ Restored original forward



In [81]:
import os
if os.path.exists(ONNX_PATH):
    file_size_mb = os.path.getsize(ONNX_PATH) / (1024**2)
    print(f"File size: {file_size_mb:.2f} MB")

    # Load and verify with ONNX
    import onnx
    try:
        onnx_model = onnx.load(str(ONNX_PATH))
        onnx.checker.check_model(onnx_model)
        print("ONNX model is valid")

        # Check LSTM nodes
        lstm_nodes = [n for n in onnx_model.graph.node if n.op_type == "LSTM"]
        print(f"\nFound {len(lstm_nodes)} LSTM node(s)")

        for i, lstm_node in enumerate(lstm_nodes):
            print(f"\nLSTM #{i+1}: {lstm_node.name}")
            print(f"  Number of inputs: {len(lstm_node.input)}")
            for j, inp in enumerate(lstm_node.input):
                if inp:  # Only print non-empty inputs
                    print(f"    [{j}] {inp}")

        # Verify with ONNX Runtime
        try:
            import onnxruntime as ort
            sess = ort.InferenceSession(str(ONNX_PATH), providers=['CPUExecutionProvider'])

            # Test inference
            test_input = dummy_input.cpu().numpy()
            ort_output = sess.run(None, {'input': test_input})[0]

            # Compare with PyTorch
            with torch.no_grad():
                torch_output = model(dummy_input).cpu().numpy()

            max_diff = abs(torch_output - ort_output).max()
            print(f"\n🔍 Output comparison:")
            print(f"  PyTorch output shape: {torch_output.shape}")
            print(f"  ONNX output shape: {ort_output.shape}")
            print(f"  Max difference: {max_diff:.6f}")

            if max_diff < 1e-4:
                print("Outputs match! ONNX export is correct.")
            else:
                print(f" Outputs differ by {max_diff}")

        except ImportError:
            print("\nonnxruntime not installed, skipping runtime verification")
        except Exception as e:
            print(f"\n ONNX Runtime verification failed: {e}")

    except Exception as e:
        print(f"ONNX verification failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("ONNX file not found!")

print("\n" + "="*60)
print("ONNX EXPORT COMPLETED")

File size: 14.52 MB
ONNX model is valid

Found 2 LSTM node(s)

LSTM #1: /bilstm/LSTM/LSTM
  Number of inputs: 7
    [0] /bilstm/LSTM/Transpose_output_0
    [1] onnx::LSTM_487
    [2] onnx::LSTM_488
    [3] onnx::LSTM_486
    [5] /bilstm/LSTM/Slice_output_0
    [6] /bilstm/LSTM/Slice_1_output_0

LSTM #2: /bilstm/LSTM/LSTM_1
  Number of inputs: 7
    [0] /bilstm/LSTM/Reshape_output_0
    [1] onnx::LSTM_530
    [2] onnx::LSTM_531
    [3] onnx::LSTM_529
    [5] /bilstm/LSTM/Slice_2_output_0
    [6] /bilstm/LSTM/Slice_3_output_0

🔍 Output comparison:
  PyTorch output shape: (1, 2)
  ONNX output shape: (1, 2)
  Max difference: 0.000000
Outputs match! ONNX export is correct.

ONNX EXPORT COMPLETED


In [82]:
DATASET_PATH = Path('/content/drive/MyDrive/AutomaticHeartSoundClassification-main/data')
sys.path.append(str(DATASET_PATH))

features_file = DATASET_PATH / "logmel_features.h5"
label_file = DATASET_PATH / "label.csv"

# Load labels để biết số sample
labels = pd.read_csv(label_file)
num_samples = len(labels)

# Load một phần features làm representative dataset (100-300 samples là đủ)
def representative_data_gen():
    with h5py.File(features_file, 'r') as hf:
        for i in range(300):  # Dùng 300 samples để calibrate
            idx = i % num_samples
            key = list(hf.keys())[idx]  # Hoặc hf['features'][idx] nếu là array
            data = hf[key][()]
            data = np.expand_dims(data, axis=0)  # Add batch dim
            data = data.astype(np.float32)
            yield [data]

print("✅ Đã chuẩn bị representative dataset từ logmel_features.h5")

✅ Đã chuẩn bị representative dataset từ logmel_features.h5


# CONVERT ONNX -> TFLITE

## ONXX

In [83]:
import onnx
m = onnx.load(ONNX_PATH)
print("IR version:", m.ir_version)

IR version: 8


In [84]:
import onnx

from onnxsim import simplify

model = onnx.load(ONNX_PATH)
model_simp, check = simplify(model)

onnx.save(model_simp, f"{MODEL_NAME.lower()}_{TIMESTAMP}_final.onnx")

In [85]:
import onnxruntime as ort
try:
    # Thử nạp model bằng ONNX Runtime
    session = ort.InferenceSession(str(ONNX_PATH))
    print("✅ Model ONNX hợp lệ và sẵn sàng chuyển đổi TFLite.")
except Exception as e:
    print(f"❌ Model vẫn lỗi cấu trúc: {e}")

✅ Model ONNX hợp lệ và sẵn sàng chuyển đổi TFLite.


In [86]:
import h5py
import numpy as np
import librosa
from pathlib import Path

PROJECT_PATH = Path('/content/drive/MyDrive/AutomaticHeartSoundClassification-main')
DATASET_PATH = PROJECT_PATH / "data"
features_file = DATASET_PATH / "logmel_features.h5"

DURATION = 5
HOP_LENGTH_MS = 15
FIXED_LENGTH = int(DURATION * 1000 / HOP_LENGTH_MS)  # 333
DELTA = True
NORM = True

def standard_normal_variate(data):
    mean = np.mean(data)
    std = np.std(data) + 1e-8
    return (data - mean) / std

calib_data_list = []

with h5py.File(features_file, 'r') as hf:
    all_keys = list(hf.keys())
    num_samples = len(all_keys)
    print(f"Total samples: {num_samples}")

    for i in range(300):
        key = all_keys[i % num_samples]
        feature = hf[key][()]

        if NORM:
            feature = standard_normal_variate(feature)

        if feature.ndim == 2:
            feature = feature[np.newaxis, :, :]

        if DELTA:
            orig = feature[0]
            delta1 = librosa.feature.delta(orig)
            delta2 = librosa.feature.delta(delta1)
            feature = np.stack([orig, delta1, delta2], axis=0)

        channels, mel_bins, num_frames = feature.shape

        if num_frames >= FIXED_LENGTH:
            start = num_frames // 2 - FIXED_LENGTH // 2
            feature = feature[:, :, start:start + FIXED_LENGTH]
        else:
            pad_width = FIXED_LENGTH - num_frames
            feature = np.pad(feature, ((0,0), (0,0), (0, pad_width)), mode='wrap')

        if channels > 1:
            feature = feature.reshape(channels * mel_bins, FIXED_LENGTH)  # (384, 333)

        feature = feature.astype(np.float32)
        feature = np.expand_dims(feature, axis=0)  # (1, 384, 333)
        calib_data_list.append(feature)

        if (i + 1) % 50 == 0:
            print(f"Prepared {i + 1}/300 calibration samples")

# Ghép thành array (300, 1, 384, 333) rồi lưu .npy
calibration_data = np.concatenate(calib_data_list, axis=0)
np.save("calib_data.npy", calibration_data)
print("✅ Đã lưu file calib_data.npy (shape:", calibration_data.shape, ")")

Total samples: 3240
Prepared 50/300 calibration samples
Prepared 100/300 calibration samples
Prepared 150/300 calibration samples
Prepared 200/300 calibration samples
Prepared 250/300 calibration samples
Prepared 300/300 calibration samples
✅ Đã lưu file calib_data.npy (shape: (300, 384, 333) )


In [42]:
import onnx
from pathlib import Path

PROJECT_PATH = Path('/content/drive/MyDrive/AutomaticHeartSoundClassification-main')
TFLITE_DIR = PROJECT_PATH / "TFLITE"

model_prefix = f"{MODEL_NAME.lower()}_{TIMESTAMP}"

onnx_files = (
    list(TFLITE_DIR.glob(f"{model_prefix}_final.onnx")) +
    list(TFLITE_DIR.glob(f"{model_prefix}.onnx"))
)

if not onnx_files:
    raise FileNotFoundError(f"Không tìm thấy ONNX cho {model_prefix}")

# Lấy file mới nhất (phòng trường hợp export nhiều lần)
onnx_path = max(onnx_files, key=lambda p: p.stat().st_mtime)

print(f"Đang dùng ONNX: {onnx_path.name}")

model = onnx.load(str(onnx_path))
input_name = model.graph.input[0].name
print(f"Tên input OP trong ONNX: '{input_name}'")

Đang dùng ONNX: crnn_0108_105831_final.onnx
Tên input OP trong ONNX: 'input'


In [87]:
from pathlib import Path
import onnx2tf

# =========================
# 1. PATH CONFIGURATION
# =========================

OUTPUT_TFLITE_DIR = TFLITE_DIR / f"{MODEL_NAME.lower()}_tflite_int8_{TIMESTAMP}"
output_folder_path=str(OUTPUT_TFLITE_DIR),

onnx_files = (
    list(TFLITE_DIR.glob(f"{MODEL_NAME.lower()}_{TIMESTAMP}_final.onnx")) +
    list(TFLITE_DIR.glob(f"{MODEL_NAME.lower()}_{TIMESTAMP}.onnx"))
)

if not onnx_files:
    raise FileNotFoundError(f"Không tìm thấy file ONNX trong {TFLITE_DIR}")

onnx_path = sorted(
    onnx_files,
    key=lambda p: p.stat().st_mtime,
    reverse=True
)[0]

print(f"Đang dùng ONNX: {onnx_path.name}")

Đang dùng ONNX: crnn_0108_105831_final.onnx


In [88]:
json_path = OUTPUT_TFLITE_DIR / f"{onnx_path.stem}_auto.json"

print(f"Kiểm tra file cấu hình: {json_path}")

Kiểm tra file cấu hình: /content/drive/MyDrive/AutomaticHeartSoundClassification-main/TFLITE/crnn_tflite_int8_0108_105831/crnn_0108_105831_final_auto.json


In [45]:

# 2. ONNX → TFLITE INT8
# CNN and VGG
'''
onnx2tf.convert(
    input_onnx_file_path=str(onnx_path),
    output_folder_path=(OUTPUT_TFLITE_DIR),

    param_replacement_file=str(json_path) if json_path.exists() else None,

    # --- TFLite settings ---
    copy_onnx_input_output_names_to_tflite=True,
    output_integer_quantized_tflite=True,
    quant_type="per-tensor",   # có thể đổi sang "per-channel"

    # --- FIX SHAPE (RẤT QUAN TRỌNG) ---
    batch_size=1,
    overwrite_input_shape=[
        "input:1,384,333"   # (B, C*Mel, Time)
    ],

    # --- INT8 CALIBRATION DATA ---
    custom_input_op_name_np_data_path=[
        [
            "input",            # tên input trong ONNX
            "calib_data.npy",   # (300, 384, 333)
            0.0,                # mean
            1.0                 # std
        ]
    ],

    non_verbose=False
)

print("Chuyển đổi thành công sang TFLite INT8!")
print("Output nằm trong thư mục: vgg11_tflite_int8_final/")
print(
    f"Output nằm trong thư mục: "
    f"{MODEL_NAME.lower()}_tflite_int8_{TIMESTAMP}/"
)
'''

'\nonnx2tf.convert(\n    input_onnx_file_path=str(onnx_path),\n    output_folder_path=(OUTPUT_TFLITE_DIR),\n\n    param_replacement_file=str(json_path) if json_path.exists() else None,\n\n    # --- TFLite settings ---\n    copy_onnx_input_output_names_to_tflite=True,\n    output_integer_quantized_tflite=True,\n    quant_type="per-tensor",   # có thể đổi sang "per-channel"\n\n    # --- FIX SHAPE (RẤT QUAN TRỌNG) ---\n    batch_size=1,\n    overwrite_input_shape=[\n        "input:1,384,333"   # (B, C*Mel, Time)\n    ],\n\n    # --- INT8 CALIBRATION DATA ---\n    custom_input_op_name_np_data_path=[\n        [\n            "input",            # tên input trong ONNX\n            "calib_data.npy",   # (300, 384, 333)\n            0.0,                # mean\n            1.0                 # std\n        ]\n    ],\n\n    non_verbose=False\n)\n\nprint("Chuyển đổi thành công sang TFLite INT8!")\nprint("Output nằm trong thư mục: vgg11_tflite_int8_final/")\nprint(\n    f"Output nằm trong thư mục:

In [91]:
import onnx
from pathlib import Path

# ============================================================
# LOAD ONNX MODEL
# ============================================================
ONNX_PATH = PROJECT_PATH / "TFLITE" / f"{MODEL_NAME.lower()}_{TIMESTAMP}_final.onnx"
MODIFIED_ONNX_PATH = PROJECT_PATH / "TFLITE" / f"{MODEL_NAME.lower()}_{TIMESTAMP}_no_init_state.onnx"

print("="*60)
print("REMOVING LSTM INITIAL STATES FROM ONNX")
print("="*60)
print(f"\nLoading: {ONNX_PATH.name}")

model = onnx.load(str(ONNX_PATH))

# ============================================================
# FIND AND MODIFY LSTM NODES
# ============================================================
lstm_nodes_found = 0
lstm_nodes_modified = 0

for node in model.graph.node:
    if node.op_type == "LSTM":
        lstm_nodes_found += 1

        print(f"\n📍 Found LSTM: {node.name}")
        print(f"   Inputs before: {len(node.input)}")

        # Show all inputs
        for i, inp in enumerate(node.input):
            if inp:
                print(f"     [{i}] {inp}")
            else:
                print(f"     [{i}] (empty)")

        # LSTM standard inputs are:
        # [0] X - input tensor
        # [1] W - weight tensor
        # [2] R - recurrence weight tensor
        # [3] B - bias tensor
        # [4] sequence_lens - (optional, can be empty)
        # [5] initial_h - (REMOVE THIS)
        # [6] initial_c - (REMOVE THIS)

        # Keep only first 5 inputs (X, W, R, B, sequence_lens)
        original_inputs = list(node.input)

        # If there are more than 5 inputs, remove the extra ones
        if len(original_inputs) > 5:
            # Keep inputs 0-4, remove 5 and 6 (initial states)
            node.input[:] = original_inputs[:5]
            lstm_nodes_modified += 1

            print(f"   ✅ Modified to {len(node.input)} inputs")
            print(f"   Removed inputs: {original_inputs[5:]}")
        else:
            print(f"   ℹ️  Already has {len(node.input)} inputs (no modification needed)")

print(f"\n📊 Summary:")
print(f"   LSTM nodes found: {lstm_nodes_found}")
print(f"   LSTM nodes modified: {lstm_nodes_modified}")

# ============================================================
# SAVE MODIFIED ONNX
# ============================================================
print(f"\n💾 Saving modified model to: {MODIFIED_ONNX_PATH.name}")

onnx.save(model, str(MODIFIED_ONNX_PATH))

# Verify the saved model
try:
    onnx.checker.check_model(model)
    print("✅ Modified ONNX model is valid\n")
except Exception as e:
    print(f"❌ Model validation failed: {e}\n")

# ============================================================
# VERIFY MODIFICATIONS
# ============================================================
print("="*60)
print("VERIFICATION")
print("="*60)

# Reload and check
model_verify = onnx.load(str(MODIFIED_ONNX_PATH))

for node in model_verify.graph.node:
    if node.op_type == "LSTM":
        num_inputs = len([inp for inp in node.input if inp])
        print(f"\nLSTM: {node.name}")
        print(f"  Total inputs: {num_inputs}")

        for i, inp in enumerate(node.input):
            if inp:
                print(f"    [{i}] {inp}")

        if num_inputs <= 5:
            print(f"  ✅ SUCCESS! No initial states")
        else:
            print(f"  ⚠️  Still has {num_inputs} inputs")

# ============================================================
# TEST WITH ONNX RUNTIME
# ============================================================
try:
    import onnxruntime as ort

    print("\n🧪 Testing with ONNX Runtime...")

    sess = ort.InferenceSession(
        str(MODIFIED_ONNX_PATH),
        providers=['CPUExecutionProvider']
    )

    # Test inference
    import numpy as np
    test_input = np.random.randn(1, 384, 333).astype(np.float32)
    output = sess.run(None, {'input': test_input})[0]

    print(f"✅ ONNX Runtime test successful")
    print(f"   Input shape: {test_input.shape}")
    print(f"   Output shape: {output.shape}")

except ImportError:
    print("\n⚠️  onnxruntime not installed, skipping runtime test")
except Exception as e:
    print(f"\n❌ ONNX Runtime test failed: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*60)
print("✅ ONNX MODIFICATION COMPLETE")
print("="*60)
print(f"\nUse this file for TFLite conversion:")
print(f"  {MODIFIED_ONNX_PATH.name}")

REMOVING LSTM INITIAL STATES FROM ONNX

Loading: crnn_0108_105831_final.onnx

📍 Found LSTM: /bilstm/LSTM/LSTM
   Inputs before: 7
     [0] /bilstm/LSTM/Transpose_output_0
     [1] onnx::LSTM_487
     [2] onnx::LSTM_488
     [3] onnx::LSTM_486
     [4] (empty)
     [5] /bilstm/LSTM/Slice_output_0
     [6] /bilstm/LSTM/Slice_output_0
   ✅ Modified to 5 inputs
   Removed inputs: ['/bilstm/LSTM/Slice_output_0', '/bilstm/LSTM/Slice_output_0']

📍 Found LSTM: /bilstm/LSTM/LSTM_1
   Inputs before: 7
     [0] /bilstm/LSTM/Reshape_output_0
     [1] onnx::LSTM_530
     [2] onnx::LSTM_531
     [3] onnx::LSTM_529
     [4] (empty)
     [5] /bilstm/LSTM/Slice_output_0
     [6] /bilstm/LSTM/Slice_output_0
   ✅ Modified to 5 inputs
   Removed inputs: ['/bilstm/LSTM/Slice_output_0', '/bilstm/LSTM/Slice_output_0']

📊 Summary:
   LSTM nodes found: 2
   LSTM nodes modified: 2

💾 Saving modified model to: crnn_0108_105831_no_init_state.onnx
✅ Modified ONNX model is valid

VERIFICATION

LSTM: /bilstm/LSTM/LS

In [95]:
import onnx
from pathlib import Path

ONNX_PATH = PROJECT_PATH / "TFLITE" / f"{MODEL_NAME.lower()}_{TIMESTAMP}_final.onnx"
MODIFIED_ONNX_PATH = PROJECT_PATH / "TFLITE" / f"{MODEL_NAME.lower()}_{TIMESTAMP}_no_init_state.onnx"

print("Đang xóa initial states từ LSTM...\n")

model = onnx.load(str(ONNX_PATH))

for node in model.graph.node:
    if node.op_type == "LSTM":
        print(f"LSTM: {node.name}")
        print(f"  Inputs trước: {len(node.input)}")

        original_inputs = list(node.input)
        if len(original_inputs) > 5:
            node.input[:] = original_inputs[:5]
            print(f"  Inputs sau: {len(node.input)}")
            print(f"  ✅ Đã xóa: {original_inputs[5:]}\n")

onnx.save(model, str(MODIFIED_ONNX_PATH))
print(f"✅ Lưu: {MODIFIED_ONNX_PATH.name}")

Đang xóa initial states từ LSTM...

LSTM: /bilstm/LSTM/LSTM
  Inputs trước: 7
  Inputs sau: 5
  ✅ Đã xóa: ['/bilstm/LSTM/Slice_output_0', '/bilstm/LSTM/Slice_output_0']

LSTM: /bilstm/LSTM/LSTM_1
  Inputs trước: 7
  Inputs sau: 5
  ✅ Đã xóa: ['/bilstm/LSTM/Slice_output_0', '/bilstm/LSTM/Slice_output_0']

✅ Lưu: crnn_0108_105831_no_init_state.onnx


In [101]:
onnx2tf.convert(
    input_onnx_file_path=str(MODIFIED_ONNX_PATH),
    output_folder_path=(OUTPUT_TFLITE_DIR),

    param_replacement_file=str(json_path) if json_path.exists() else None,

    # --- TFLite settings ---
    copy_onnx_input_output_names_to_tflite=True,
    output_integer_quantized_tflite=True,
    quant_type="per-tensor",   # có thể đổi sang "per-channel"

    # --- FIX SHAPE (RẤT QUAN TRỌNG) ---
    batch_size=1,
    overwrite_input_shape=[
        "input:1,384,333"   # (B, C*Mel, Time)
    ],

    # --- INT8 CALIBRATION DATA ---
    custom_input_op_name_np_data_path=[
        [
            "input",            # tên input trong ONNX
            "calib_data.npy",   # (300, 384, 333)
            0.0,                # mean
            1.0                 # std
        ]
    ],

    non_verbose=False
)

print("Chuyển đổi thành công sang TFLite INT8!")
print("Output nằm trong thư mục: vgg11_tflite_int8_final/")
print(
    f"Output nằm trong thư mục: "
    f"{MODEL_NAME.lower()}_tflite_int8_{TIMESTAMP}/"
)


Model optimizing started ============================================================
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃            ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Constant   │ 27             │ 27               │
│ Conv       │ 8              │ 8                │
│ Gather     │ 1              │ 1                │
│ LSTM       │ 2              │ 2                │
│ MatMul     │ 1              │ 1                │
│ MaxPool    │ 5              │ 5                │
│ Relu       │ 8              │ 8                │
│ Reshape    │ 3              │ 3                │
│ Squeeze    │ 1              │ 1                │
│ Transpose  │ 4              │ 4                │
│ Model Size │ 14.5MiB        │ 14.5MiB          │
└────────────┴────────────────┴──────────────────┘

Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ 

TypeError: 'NoneType' object is not subscriptable

# Test Model

In [102]:
OUTPUT_TFLITE_DIR = TFLITE_DIR / f"{MODEL_NAME.lower()}_tflite_int8_{TIMESTAMP}"

In [ ]:
from pathlib import Path

output_dir = Path(OUTPUT_TFLITE_DIR)
print("Các file/thư mục trong output:")
for item in output_dir.iterdir():
    print(item.name)

In [ ]:
import tensorflow as tf
from pathlib import Path


print(f"Đang kiểm tra thư mục: {OUTPUT_TFLITE_DIR}\n")

# Kiểm tra xem thư mục có tồn tại không
if not OUTPUT_TFLITE_DIR.exists():
    raise FileNotFoundError(f"Thư mục không tồn tại: {OUTPUT_TFLITE_DIR}\n"
                            "Hãy kiểm tra lại tên thư mục hoặc chạy lại phần chuyển đổi ONNX → TFLite.")

# Liệt kê tất cả file .tflite trong thư mục
tflite_files = list(OUTPUT_TFLITE_DIR.glob("*.tflite"))

if not tflite_files:
    raise FileNotFoundError(f"Không tìm thấy file .tflite nào trong {OUTPUT_TFLITE_DIR}")

# In danh sách file kèm kích thước
print("=== DANH SÁCH FILE TFLITE & KÍCH THƯỚC ===")
def format_size(size_bytes):
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size_bytes < 1024:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024
    return f"{size_bytes:.2f} TB"

for file_path in sorted(tflite_files):
    size = file_path.stat().st_size
    print(f"{file_path.name:<55} {format_size(size)}")

print()  # dòng trống

# Chọn file INT8 ưu tiên (có chứa "integer", "int8", "quant")
int8_file = next(
    (f for f in tflite_files if any(keyword in f.name.lower() for keyword in ["integer", "int8", "quant"])),
    tflite_files[0]  # nếu không có thì dùng file đầu tiên
)

print(f"Đang test với: {int8_file.name} ({format_size(int8_file.stat().st_size)})\n")

# Load và test model
interpreter = tf.lite.Interpreter(model_path=str(int8_file))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("=== THÔNG TIN INPUT ===")
for i, detail in enumerate(input_details):
    print(f"Input {i}:")
    print(f"  Name: {detail['name']}")
    print(f"  Shape: {detail['shape']}")
    print(f"  Dtype: {detail['dtype']}")
    print(f"  Quantization: {detail['quantization']}")

print("\n=== THÔNG TIN OUTPUT ===")
for detail in output_details:
    print(f"Output shape: {detail['shape']}")

In [ ]:
import h5py
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import librosa
from pathlib import Path

PROJECT_PATH = Path('/content/drive/MyDrive/AutomaticHeartSoundClassification-main')
DATASET_PATH = PROJECT_PATH / "data"

features_file = DATASET_PATH / "logmel_features.h5"
label_file = DATASET_PATH / "label.csv"

# ================== LOGMEL DATASET ==================
EPS = 1e-8

def standard_normal_variate(data):
    mean = np.mean(data)
    std = np.std(data) + EPS
    return (data - mean) / std

class LogMelDataset(Dataset):
    def __init__(self, features_h5_path, label_csv_path, keys=None,
                 delta=True, norm=True, duration=5, hop_length=15, training=False):
        self.features_h5 = h5py.File(features_h5_path, 'r')
        self.labels_df = pd.read_csv(label_csv_path)

        if keys is None:
            self.keys = list(self.features_h5.keys())
        else:
            self.keys = keys

        # Map filename (stem) → label
        self.key_to_label = dict(zip(self.labels_df.iloc[:, 0].astype(str), self.labels_df.iloc[:, 1]))

        self.delta = delta
        self.norm = norm
        self.duration = duration
        self.hop_length = hop_length
        self.training = training
        self.fixed_length = int(self.duration * 1000 / self.hop_length)  # 333 frames

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        feature = self.features_h5[key][()]  # (mel_bins, time_frames)

        if self.norm:
            feature = standard_normal_variate(feature)

        if feature.ndim == 2:
            feature = feature[np.newaxis, :, :]  # (1, mel, T)

        channels, mel_bins, num_frames = feature.shape

        if self.delta:
            orig = feature[0]
            delta1 = librosa.feature.delta(orig)
            delta2 = librosa.feature.delta(delta1)
            feature = np.stack([orig, delta1, delta2], axis=0)  # (3, mel, T)
            channels = 3

        # Crop/pad to fixed 333 frames
        if num_frames >= self.fixed_length:
            start = (num_frames - self.fixed_length) // 2
            feature = feature[:, :, start:start + self.fixed_length]
        else:
            pad_width = self.fixed_length - num_frames
            feature = np.pad(feature, ((0,0),(0,0),(0,pad_width)), mode='wrap')

        # Ghép 3 channel → 384 mel bins (như trong model VGG11)
        if channels > 1:
            feature = feature.reshape(channels * mel_bins, self.fixed_length)  # (384, 333)

        feature_tensor = torch.from_numpy(feature).float()  # (384, 333)

        label = torch.tensor(self.key_to_label.get(key, 0), dtype=torch.long)
        return feature_tensor, label

    def close(self):
        self.features_h5.close()

In [ ]:
# Load toàn bộ keys
with h5py.File(features_file, 'r') as hf:
    all_keys = list(hf.keys())

print(f"Tổng số mẫu: {len(all_keys)}")

# Tạo dataset toàn bộ (dùng để test cuối cùng)
full_dataset = LogMelDataset(
    features_file,
    label_file,
    keys=all_keys,
    delta=True,
    norm=True,
    duration=5,
    hop_length=15,
    training=False  # không random crop
)

full_loader = DataLoader(
    full_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print("✅ Đã tạo DataLoader thành công!")

In [ ]:
import tensorflow as tf
import numpy as np

def test_tflite_accuracy(tflite_path, data_loader):
    # Load TFLite model
    interpreter = tf.lite.Interpreter(model_path=str(tflite_path))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_shape = input_details[0]['shape']  # ví dụ [1, 333, 384] hoặc [1, 384, 333]

    print(f"Testing model: {tflite_path.name}")
    print(f"Expected input shape: {input_shape}")

    correct = 0
    total = 0

    for features, labels in data_loader:
        batch_size = features.shape[0]

        for i in range(batch_size):
            x = features[i].numpy()  # (384, 333)

            # Điều chỉnh shape theo model TFLite yêu cầu
            if input_shape[1] == 333 and input_shape[2] == 384:
                x = x.transpose(1, 0)  # (333, 384)
            # Nếu là (1, 384, 333) → giữ nguyên

            x = np.expand_dims(x, axis=0).astype(np.float32)  # (1, H, W)

            interpreter.set_tensor(input_details[0]['index'], x)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])  # (1, 2)

            pred = np.argmax(output, axis=1)[0]
            true_label = labels[i].item()

            if pred == true_label:
                correct += 1
            total += 1

    accuracy = correct / total
    print(f"=> Accuracy trên {total} mẫu: {accuracy*100:.2f}% ({correct}/{total})")
    print("-" * 60)
    return accuracy

In [ ]:
from pathlib import Path

# Tìm file INT8
tflite_files = list(OUTPUT_TFLITE_DIR.glob("*integer*.tflite")) + list(OUTPUT_TFLITE_DIR.glob("*quant*.tflite"))
if not tflite_files:
    raise FileNotFoundError("Không tìm thấy file TFLite INT8!")

int8_tflite_path = tflite_files[0]
print(f"Đang test với: {int8_tflite_path.name}")

# Chạy test
test_tflite_accuracy(int8_tflite_path, full_loader)

# Đóng dataset
full_dataset.close()